### A VAGAN and a MCI/AD classifier are trained separately. Then the classifier is applied to the raw samples and the VAGAN preprocessed samples.

In [ ]:
import os
os.chdir('/local/mhoerold/entrack')

In [ ]:
import yaml
import copy
import pydoc
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from src.test_retest.mri.supervised_features import SliceClassification
from src.data.streaming.base import Group

## Some constants

In [ ]:
vagan_label = "mri_slice_iterated"
clf_label = "clf_mci_ad"
conversion_key = "mci_ad_conv_delta_2"
test_file = "data/20180905-081310-debug/test.txt"
clf_folder = os.path.join("data", clf_label)
with open(os.path.join(clf_folder, "config.yaml"), 'r') as f:
    clf_config = yaml.load(f)

## Load classifier only

In [ ]:
clf_only_config = copy.deepcopy(clf_config)
clf_only_config["params"]["streamer"]["class"] = "src.data.streaming.mri_streaming.MRIConversionSingleStream"
clf_only_config["params"]["streamer"]["class"] = pydoc.locate(clf_only_config["params"]["streamer"]["class"])
clf_only_config["params"]["streamer"]["params"]["stream_config"]["conversion_delta"] = 3

clf_only_obj = SliceClassification(**clf_only_config["params"])
clf_only_est = tf.estimator.Estimator(
    model_fn=clf_only_obj.model_fn,
    model_dir=clf_folder,
    params=clf_only_config["params"]["params"]
)

## Load classifier with VAGAN preprocessing

In [ ]:
clf_vagan_config = copy.deepcopy(clf_config)
clf_vagan_config["params"]["streamer"]["class"] = "src.data.streaming.vagan_preprocessing.VaganConversionFarPredictions"
clf_vagan_config["params"]["streamer"]["class"] = pydoc.locate(clf_vagan_config["params"]["streamer"]["class"])
clf_vagan_config["params"]["streamer"]["params"]["stream_config"]["conversion_delta"] = 3
clf_vagan_config["params"]["streamer"]["params"]["stream_config"]["vagan_steps"] = 3
clf_vagan_config["params"]["streamer"]["params"]["stream_config"]["vagan_label"] = vagan_label
clf_vagan_config["params"]["streamer"]["params"]["stream_config"]["cache_preprocessing"] = vagan_label

clf_vagan_obj = SliceClassification(**clf_vagan_config["params"])
clf_vagan_est = tf.estimator.Estimator(
    model_fn=clf_vagan_obj.model_fn,
    model_dir=clf_folder,
    params=clf_vagan_config["params"]["params"]
)

## Load test data

In [ ]:
file_ids = []
with open(test_file, 'r') as f:
    for line in f:
        fid = line.strip()
        file_ids.append(fid)
        
file_ids = clf_only_obj.streamer.select_file_ids(file_ids)

In [ ]:
# labels
labels = [clf_only_obj.streamer.get_meta_info_by_key(fid, conversion_key) for fid in file_ids]
batches = [Group([fid]) for fid in file_ids]

clf_input_fn = clf_only_obj.streamer.get_input_fn_for_groups(batches)

vagan_input_fn = clf_vagan_obj.streamer.get_input_fn_for_groups(batches)

In [ ]:
def predict_probabilities(est, input_fn):
    preds = est.predict(input_fn, ["probs"])
    res = []
    for pred in preds:
        res.append(pred["probs"][1])  ## probability of being AD
        
    return np.array(res)

In [ ]:
t0_probs = predict_probabilities(clf_only_est, clf_input_fn)

In [ ]:
vagan_probs = predict_probabilities(clf_vagan_est, vagan_input_fn)

In [ ]:
def threshold_diff(labels, t0_probs, vagan_probs):
    all_eps = np.linspace(-1, 1, 200)
    
    expected = np.array(labels)
    diffs = vagan_probs - t0_probs
    accs = []
    for eps in all_eps:
        predicted_conv = (diffs > eps).astype(np.float32)
        
        acc = np.mean(predicted_conv == expected)
        accs.append(acc)
    
    print(np.max(accs))
    plt.figure()
    plt.plot(all_eps, accs, marker='o')
    plt.show()
    
def threshold_vagan_prob(labels, vagan_probs):
    all_eps = np.linspace(-1, 1, 200)
    
    expected = np.array(labels)
    accs = []
    for eps in all_eps:
        predicted_conv = (vagan_probs > eps).astype(np.float32)
        
        acc = np.mean(predicted_conv == expected)
        accs.append(acc)
    
    print(np.max(accs))
    plt.figure()
    plt.plot(all_eps, accs, marker='o')
    plt.show()

In [ ]:
threshold_diff(labels, t0_probs, vagan_probs)

In [ ]:
threshold_vagan_prob(labels, vagan_probs)